# Preprocessing dataset

In [77]:
import re
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from tqdm import tqdm_notebook as tqdm
from pathlib import Path


def check_file_exist(file_path):
    my_file = Path(file_path)
    if my_file.is_file():
        return True
    else:
        return False


def read_data():
    train = pd.read_fwf(data_folder + "train_tweets.txt", infer_nrows=150, sep="\t", header=None,
                        names=["UID", "Twitter"])
    train = train.dropna()
    test = pd.read_fwf(data_folder + "test_tweets_unlabeled.txt", infer_nrows=150, header=None, names=["Twitter"])
    return train, test


def regularize(text, tk):
    tokens = tk.tokenize(text)
    for i,token in enumerate(tokens):
        if token.find("http") != -1:
            token = re.sub(r"http://", "{",token)
            token = re.sub(r"/[\w./-]+", "}", token)
            tokens[i]=token
    return tokens


def stemming(tokens, stemmer):
    return [stemmer.stem(token).lower() for token in tokens]


def pre_processing(df, description):
    stemmed=[]
    tk = TweetTokenizer()
    stemmer = PorterStemmer()
    print("Start normalizing {} set".format(description))
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        tokens = regularize(row["Twitter"], tk)
        stemmed.append("\x01".join(stemming(tokens, stemmer)))
    del df["Twitter"]
    df["Twitter"] = stemmed
    df.to_csv(data_folder + "{}_norm.csv".format(description),index=False)
    return df

# Specify data folder
data_folder = "./"

train, test = read_data()
if check_file_exist(data_folder+"train_norm.csv"):
    print("tain_norm.txt exists. Will load it")
    train_norm = pd.read_csv(data_folder+"train_norm.csv")
else:
    train_norm = pre_processing(train, "train")
if check_file_exist(data_folder+"test_norm.csv"):
    print("test_norm.txt exists. Will load it")
    test_norm = pd.read_csv(data_folder+"test_norm.csv")
else:
    test_norm = pre_processing(test, "test")


tain_norm.txt exists. Will load it
test_norm.txt exists. Will load it


# Vectorize data using BOW

In [78]:
from sklearn.feature_extraction import DictVectorizer

def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW

def prepare_data(feature_extractor):
    training_set = []
    test_set = []
    training_classifications = []
    for _, row in tqdm(train_norm.iterrows(),total = train_norm.shape[0]):
        feature_dict = feature_extractor(row["Twitter"].split("\x01"))   
        training_set.append(feature_dict)
        training_classifications.append(row["UID"])
    for _, row in tqdm(test_norm.iterrows(),total = test_norm.shape[0]):
        features = feature_extractor(row["Twitter"].split("\x01"))
        test_set.append(features)
    vectorizer = DictVectorizer()
    training_data = vectorizer.fit_transform(training_set)
    test_data = vectorizer.transform(test_set)
    return training_data,training_classifications,test_data

# trn_data,trn_classes,test_data = prepare_data(get_BOW)

# Vectorize data using TFIDF

In [79]:
from sklearn.feature_extraction.text import TfidfTransformer

def prepare_data_tfidf(feature_extractor):
    training_set = []
    test_set = []
    training_classifications = []
    for _, row in tqdm(train_norm.iterrows(),total = train_norm.shape[0]):
        feature_dict = feature_extractor(row["Twitter"].split("\x01"))   
        training_set.append(feature_dict)
        training_classifications.append(row["UID"])

    for _, row in tqdm(test_norm.iterrows(),total = test_norm.shape[0]):
        features = feature_extractor(row["Twitter"].split("\x01"))
        test_set.append(features)
        
    vectorizer = DictVectorizer()
    transformer = TfidfTransformer(smooth_idf=False,norm=None)
    
    training_data = transformer.fit_transform(vectorizer.fit_transform(training_set))
    test_data  = transformer.transform(vectorizer.transform(test_set))
    return training_data,training_classifications,test_data

trn_data,trn_classes,test_data = prepare_data_tfidf(get_BOW)

In [80]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

clfs = [KNeighborsClassifier(n_jobs=-1),DecisionTreeClassifier(),RandomForestClassifier(n_jobs=-1, n_estimators=50),
        MultinomialNB(),LinearSVC(verbose=10),LogisticRegression(n_jobs=-1)]


# Just for comparing the perforamnce of different models
High memory requirement

In [81]:
from sklearn import model_selection
#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from time import time
import numpy as np

def do_multiple_10foldcrossvalidation(clfs,data,classifications):
    stacking_train_data=[]
    for clf in clfs:
        s_time = time()
        predictions = model_selection.cross_val_predict(clf, data, classifications, cv=10)
        stacking_train_data.append(predictions)
        print (clf)
        print ("accuracy")
        print (accuracy_score(classifications,predictions))
        print (classification_report(classifications,predictions))
        print("time cost:{}".format(time() - s_time))
    return np.transpose(stacking_train_data)

def get_cv_predictions(begin_idx, end_idx):
    if check_file_exist(data_folder+"cv_predictions_{}-{}.csv".format(begin_idx, end_idx-1)):
        print("cv_predictions_{}-{}.csv".format(begin_idx, end_idx-1)+" exists. Will load it")
        stacking_train_data = pd.read_csv(data_folder+"cv_predictions_{}-{}.csv".format(begin_idx, end_idx-1))[[str(i) for i in range(begin_idx,end_idx)]].values
    else:
        stacking_train_data = do_multiple_10foldcrossvalidation(clfs[begin_idx:end_idx],trn_data,trn_classes)
        out_df = pd.DataFrame(stacking_train_data, columns=[i for i in range(begin_idx,end_idx)])
        out_df.to_csv(data_folder + "cv_predictions_{}-{}.csv".format(begin_idx, end_idx-1))
    return stacking_train_data

get_cv_predictions(4,5) #get the cross validation prediction for model 4

# train and predict

In [ ]:
import pickle


def save_predicted(predicted, index):
    output = [(i+1,pred) for i,pred in enumerate(predicted)]
    out_df = pd.DataFrame(output, columns=["Id", "Predicted"]).set_index("Id")
    out_df.to_csv(data_folder + "predicted_{}.csv".format(index))

def fit_predict(clfs, indexs, data, classifications):
    for i in indexs:
        s_time = time()
        clfs[i].fit(data, classifications)
        save_predicted(clfs[i].predict(test_data), i)
        with open('trained_{}.pkl'.format(i), 'wb') as fid:
            pickle.dump(clfs[i], fid, protocol=4)
        print("time cost:{}".format(time() - s_time))
    
def load_predict(indexs):
    for i in indexs:
        with open('trained_{}.pkl'.format(i), 'rb') as fid:
            model = pickle.load(fid)
        save_predicted(model.predict(test_data),i)

models_index = list(range(4,5))
fit_predict(clfs, models_index, trn_data, trn_classes)
# load_predict(models_index)

[LibLinear]

# ensemble with xgboosting

In [75]:
import xgboost as xgb

def load_xgboosting_test_data(begin_idx, end_idx):
    x_test=[]
    for i in range(begin_idx, end_idx):
        x_test.append(np.array(pd.read_csv('predicted_{}.csv'.format(i))["Predicted"]))
    return np.transpose(x_test)
        
def xgboosting_fit_predict(begin_idx, end_idx):
    s_time = time()
    x_train=get_cv_predictions(begin_idx, end_idx)
    gbm = xgb.XGBClassifier(
         n_estimators= 2000,
        #  max_depth= 4,
         min_child_weight= 1,
         gamma=0.5,                        
         subsample=1,
         colsample_bytree=1,
         objective= 'multi:softmax',
         nthread= -1,
         scale_pos_weight=1).fit(x_train, trn_classes)
    
    x_test = load_xgboosting_test_data(begin_idx, end_idx)
    save_predicted(gbm.predict(x_test), "gbm_{}-{}".format(begin_idx, end_idx))
    
    with open('trained_xgboosting_{}-{}.pkl'.format(begin_idx, end_idx-1), 'wb') as fid:
        pickle.dump(gbm, fid, protocol=4)
    print("time cost:{}".format(time() - s_time))

def xgboosting_load_predict(begin_idx, end_idx):
    with open('trained_xgboosting_{}-{}.pkl'.format(begin_idx, end_idx-1), 'rb') as fid:
        model = pickle.load(fid)
    x_test = load_xgboosting_test_data(begin_idx, end_idx)
    save_predicted(model.predict(x_test),"gbm_{}-{}".format(begin_idx, end_idx-1))
    
xgboosting_fit_predict(4, 6)
# xgboosting_load_predict(4, 6)

cv_predictions_4-5.csv exists. Will load it
time cost:0.08347606658935547
